In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [5]:
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [8]:
input_sequences = []

In [9]:
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [10]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [11]:
x = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [12]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1, input_shape=(max_sequence_len-1,)))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x,y, epochs=100, verbose=1)

In [ ]:
seed_text = 'I will leave if they'
next_words = 3

In [ ]:
for _ in range(next_words):
  token_list = tokenizer.texts_to_sequeneces([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = np.argmax(model.predict(token_list), axis=-1)
  output_word = ''
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += ' ' + output_word
print(seed_text)